In [1]:
import os
import openai
import requests
import json
import sseclient
import pandas as pd

In [32]:
# openai.api_key = os.getenv("OPENAI_API_KEY")

import dotenv

env_file = '.env'
dotenv.load_dotenv(env_file, override=True)
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.Completion.create(
#   model="text-davinci-003",
#   prompt="Say this is a test",
#   max_tokens=7,
#   temperature=0
# )

In [4]:
# def send_request(prompt):
#     openai.api_key = os.getenv("OPENAI_API_KEY")
#     response = openai.Completion.create(
#       model="text-davinci-003",
#       prompt=prompt,
#       temperature=0
#     )
    
#     return response

In [5]:
# response = send_request("Say this is a test")
# response

In [6]:
# response.choices

In [13]:
real_prompt = """
You are an enthusiastic iTwinUI representative who loves helping users. iTwinUI is a UI library with a React components like `Button`, `Table`, `Header`, `Text`, etc. where all components follow the strict UX guidelines set by iTwinUI. Given the "Most relevant chunks" from the iTwinUI documentation website, give an answer in markdown (code and hierarchical bullet points, whenever possible) to the "User query". You should not use any information outside of "Most relevant chunks". If you are not confident about your answer, say "Sorry, I do not have a confident answer". NEVER guess an answer or inject your knowledge.

"User query":
Where should I place an Alert component? Like from a UX perspective 

"Most relevant chunks":
```json
[
  {
    "header": ["Alert", "Placement"],
    "content": "The alert notification can display in two different ways:"
  },
  {
    "header": ["Alert", "Placement", "Inline"],
    "content": "An inline alert appears embedded within the copy of the site as a block element. This means that when the user scrolls, the alert scrolls as well.\n\nInline alerts can optionally be dismissible. Once the alert is dismissed, the content below it will shift upwards due to the alert no longer consuming space."
  },
  {
    "header": ["Alert", "Placement", "Sticky"],
    "content": "A sticky alert appears full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible. The alert appears above all other UI elements except for toast notifications and modals.\n\nIt is up to the development team wether or not the sticky alert should push the rest of the content down or not."
  },
]
```
""".strip()

real_prompt

'You are an enthusiastic iTwinUI representative who loves helping users. iTwinUI is a UI library with a React components like `Button`, `Table`, `Header`, `Text`, etc. where all components follow the strict UX guidelines set by iTwinUI. Given the "Most relevant chunks" from the iTwinUI documentation website, give an answer in markdown (code and hierarchical bullet points, whenever possible) to the "User query". You should not use any information outside of "Most relevant chunks". If you are not confident about your answer, say "Sorry, I do not have a confident answer". NEVER guess an answer or inject your knowledge.\n\n"User query":\nWhere should I place an Alert component? Like from a UX perspective \n\n"Most relevant chunks":\n```json\n[\n  {\n    "header": ["Alert", "Placement"],\n    "content": "The alert notification can display in two different ways:"\n  },\n  {\n    "header": ["Alert", "Placement", "Inline"],\n    "content": "An inline alert appears embedded within the copy of t

In [14]:
# real_prompt_response = send_request(real_prompt)

In [36]:
def generate_streaming_text(prompt, model="text-ada-001", max_tokens=30):
    headers = {
        "Content-Type": "application/json",
        "Accept": "text/event-stream",
        "Authorization": f"Bearer {openai.api_key}"
    }
    data = {
        "prompt": prompt,
        "model": model,
        "max_tokens": max_tokens,
        "temperature": 0.0,
        "stream":True
    }

    print(data)

    response = requests.post("https://api.openai.com/v1/completions", headers=headers, data=json.dumps(data))
    # print("response", response)
    client = sseclient.SSEClient(response)
    for event in client.events():
        # print("HERE")
        if event.data != "[DONE]":
            response_json = json.loads(event.data)
            # print(response_json["choices"][0]["text"], end="", flush=True)

            if "choices" in response_json:
                # print(response_json["choices"][0]["text"], end="", flush=True)
                yield response_json["choices"][0]["text"]

        # if event.event == "data":
        #     response_json = json.loads(event.data)
        #     if "choices" in response_json:
        #         return response_json["choices"][0]["text"]
        #     else:
        #         return response_json
        
    # response_json = response.json()
    # return response_json

In [37]:
for chunk in generate_streaming_text(real_prompt, max_tokens=1000):
    print(chunk, end="", flush=True)

{'prompt': 'You are an enthusiastic iTwinUI representative who loves helping users. iTwinUI is a UI library with a React components like `Button`, `Table`, `Header`, `Text`, etc. where all components follow the strict UX guidelines set by iTwinUI. Given the "Most relevant chunks" from the iTwinUI documentation website, give an answer in markdown (code and hierarchical bullet points, whenever possible) to the "User query". You should not use any information outside of "Most relevant chunks". If you are not confident about your answer, say "Sorry, I do not have a confident answer". NEVER guess an answer or inject your knowledge.\n\n"User query":\nWhere should I place an Alert component? Like from a UX perspective \n\n"Most relevant chunks":\n```json\n[\n  {\n    "header": ["Alert", "Placement"],\n    "content": "The alert notification can display in two different ways:"\n  },\n  {\n    "header": ["Alert", "Placement", "Inline"],\n    "content": "An inline alert appears embedded within th

In [17]:
for chunk in generate_streaming_text(real_prompt, model="text-babbage-001", max_tokens=300):
    print(chunk, end="", flush=True)

In [18]:
for chunk in generate_streaming_text(real_prompt, model="text-curie-001", max_tokens=300):
    print(chunk, end="", flush=True)

In [30]:
for chunk in generate_streaming_text(real_prompt, model="text-davinci-003", max_tokens=300):
    print(chunk, end="", flush=True)



It depends on the type of alert you are displaying. 

For an inline alert, it should appear embedded within the copy of the site as a block element. This means that when the user scrolls, the alert scrolls as well.

For a sticky alert, it should appear full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible. The alert appears above all other UI elements except for toast notifications and modals. It is up to the development team wether or not the sticky alert should push the rest of the content down or not.

In [31]:
for chunk in generate_streaming_text(real_prompt, model="text-davinci-003", max_tokens=600):
    print(chunk, end="", flush=True)



It depends on the type of alert you are displaying. 

For an inline alert, it should appear embedded within the copy of the site as a block element. This means that when the user scrolls, the alert scrolls as well.

For a sticky alert, it should appear full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible. The alert appears above all other UI elements except for toast notifications and modals. It is up to the development team wether or not the sticky alert should push the rest of the content down or not.

In [20]:
real_prompt_1 = """
You are an enthusiastic iTwinUI representative who loves helping users. iTwinUI is a UI library with a React components like `Button`, `Table`, `Header`, `Text`, etc. where all components follow the strict UX guidelines set by iTwinUI. Given the "Most relevant chunks" from the iTwinUI documentation website, give an answer in markdown to the "User query". You should not use any information outside of "Most relevant chunks". If you are not confident about your answer, say "Sorry, I do not have a confident answer". NEVER guess an answer or inject your knowledge.

"User query":
Where should I place an Alert component? Like from a UX perspective 

"Most relevant chunks":
```json
[
  {
    "header": ["Alert", "Placement"],
    "content": "The alert notification can display in two different ways:"
  },
  {
    "header": ["Alert", "Placement", "Inline"],
    "content": "An inline alert appears embedded within the copy of the site as a block element. This means that when the user scrolls, the alert scrolls as well.\n\nInline alerts can optionally be dismissible. Once the alert is dismissed, the content below it will shift upwards due to the alert no longer consuming space."
  },
  {
    "header": ["Alert", "Placement", "Sticky"],
    "content": "A sticky alert appears full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible. The alert appears above all other UI elements except for toast notifications and modals.\n\nIt is up to the development team wether or not the sticky alert should push the rest of the content down or not."
  },
]

Your answer in markdown (hierarchical bullet points, whenever possible):
```
""".strip()

In [21]:
for chunk in generate_streaming_text(real_prompt_1, model="text-davinci-003", max_tokens=600):
    print(chunk, end="", flush=True)

---

Now testing embeddings

In [5]:
sample_embedding = openai.Embedding.create(
  model="text-embedding-ada-002",
  input="The food was delicious and the waiter..."
)

In [8]:
sample_embedding

<OpenAIObject list at 0x21aaaec9bd0> JSON: {
  "data": [
    {
      "embedding": [
        0.002235695719718933,
        -0.009273056872189045,
        0.0158150065690279,
        -0.007805881090462208,
        -0.004738150630146265,
        0.014824187383055687,
        -0.00978752039372921,
        -0.03828629106283188,
        -0.00685316976159811,
        -0.02870836667716503,
        0.025291308760643005,
        0.018152326345443726,
        -0.003636181354522705,
        -0.025532662868499756,
        0.0005212124669924378,
        -0.016323119401931763,
        0.028454311192035675,
        0.005335183348506689,
        0.009596978314220905,
        -0.016500959172844887,
        -0.015345003455877304,
        0.004258619621396065,
        0.007018306292593479,
        -0.00706276623532176,
        -0.0039283460937440395,
        0.01857151836156845,
        0.008688726462423801,
        -0.02273804321885109,
        0.01145158987492323,
        0.0238431878387928,
        0.0

In [17]:
sample_embedding["data"][0]["embedding"]

[0.002235695719718933,
 -0.009273056872189045,
 0.0158150065690279,
 -0.007805881090462208,
 -0.004738150630146265,
 0.014824187383055687,
 -0.00978752039372921,
 -0.03828629106283188,
 -0.00685316976159811,
 -0.02870836667716503,
 0.025291308760643005,
 0.018152326345443726,
 -0.003636181354522705,
 -0.025532662868499756,
 0.0005212124669924378,
 -0.016323119401931763,
 0.028454311192035675,
 0.005335183348506689,
 0.009596978314220905,
 -0.016500959172844887,
 -0.015345003455877304,
 0.004258619621396065,
 0.007018306292593479,
 -0.00706276623532176,
 -0.0039283460937440395,
 0.01857151836156845,
 0.008688726462423801,
 -0.02273804321885109,
 0.01145158987492323,
 0.0238431878387928,
 0.015573653392493725,
 -0.003575843060389161,
 -0.03488193452358246,
 -0.0041379425674676895,
 -0.02609158679842949,
 -0.021505869925022125,
 -0.005735321901738644,
 0.011724700219929218,
 0.008358453400433064,
 0.004131591413170099,
 0.0191685501486063,
 -0.014379588887095451,
 0.00893643219023943,
 0.

In [18]:
type(sample_embedding["data"][0]["embedding"])

list

---

Now testing with real chunks

In [28]:
def get_embeddings(input, model="text-embedding-ada-002"):
  response = openai.Embedding.create(
    model=model,
    input=input
  )
  return response

In [20]:
real_chunks = [
  {
    "header": [
      "Alert"
    ],
    "link": "/docs/alert",
    "content": "A small box to quickly grab the user's attention and communicate a brief message.\n\n\nAn alert is an element that notifies the user of something important that is not user initiated. If you need to alert the user because of something they did, consider using a toast notification or an inline message. Instead, an alert is used to notify the user of something unrelated to what they just did. Examples of this might be announcing a new feature within the product, or alerting the user of scheduled server maintenance and downtime."
  },
  {
    "header": [
      "Alert",
      "Appearance"
    ],
    "link": "/docs/alert#appearance",
    "content": "The alert message must be concise, we recommend no more than 256 characters. If you feel the need to explain in further detail, you may use an optional hyperlink at the end of the alert message to explain more on a new page. You may also include an optional close icon (close icon is required for sticky alerts)."
  },
]

In [21]:
for chunk in real_chunks:
    print(chunk)

['Alert']
['Alert', 'Appearance']


In [29]:
embeddings = []
for chunk in real_chunks:
    embedding = get_embeddings(f"{chunk}")
    embeddings.append(embedding)

In [32]:
embeddings

[<OpenAIObject list at 0x21ad3e74f40> JSON: {
   "data": [
     {
       "embedding": [
         -0.0216156505048275,
         0.03179503604769707,
         0.006474336609244347,
         -0.0014709692914038897,
         0.019847726449370384,
         -0.005738851614296436,
         -0.010876885615289211,
         -0.01807980053126812,
         -0.014378206804394722,
         -0.007838263176381588,
         -0.010593741200864315,
         0.00673330994322896,
         -0.03483365476131439,
         -0.0020752407144755125,
         -0.009081335738301277,
         0.0068438053131103516,
         0.017582571133971214,
         -0.020579757168889046,
         0.00028594984905794263,
         -0.010628270916640759,
         -0.009557846933603287,
         0.009709778241813183,
         0.002386008854955435,
         0.01843891106545925,
         -0.012244264595210552,
         0.005307229235768318,
         0.018038364127278328,
         -0.02819012477993965,
         -0.0005006820429116488

In [48]:
embeddings_df = pd.DataFrame([], columns=["chunk", "object", "embedding_object", "index", "embedding", "prompt_tokens", "total_tokens"])

i = 0
for chunk, embedding in zip(real_chunks, embeddings):
    embeddings_df.loc[len(embeddings_df)] = {
        "chunk": chunk,
        "object": embedding["object"],
        "embedding_object": embedding["data"][0]["object"],
        "index": embedding["data"][0]["index"],
        "embedding": embedding["data"][0]["embedding"],
        "prompt_tokens": embedding["usage"]["prompt_tokens"],
        "total_tokens": embedding["usage"]["total_tokens"],
    }

embeddings_df

,chunk,object,embedding_object,index,embedding,prompt_tokens,total_tokens
0,"{'header': ['Alert'], 'link': '/docs/alert', '...",list,embedding,0,"[-0.0216156505048275, 0.03179503604769707, 0.0...",122,122
1,"{'header': ['Alert', 'Appearance'], 'link': '/...",list,embedding,0,"[-0.002292848890647292, 0.04747709631919861, 0...",88,88


In [50]:
embeddings_df.to_parquet("embeddings_df.parquet")